# Czyszczenie danych i przygotowanie zbioru do analizy
Zadaniem do wykonania w tym projekcie jest oczyszczenie, a także usunięcie problemów w dostarczonym zbiorze TitatnicMess.tsv. 
Dane należy uporządkować, a także opisać napotkane problemy

Pierwszym krokiem w pracy przy zbiorze danych jest import potrzebnych bibliotek, które będą wykorzystywane podczas czyszczenia zbioru.

In [234]:
import locale
import pandas as pd
import math
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' )

Kolejnym krokiem jest wczytanie podanego zbioru danych TitanicMess.tsv z odpowiednim separatorem oraz kolumną indeksującą.

In [235]:
titanic = pd.read_csv('TitanicMess.tsv', sep='\t', index_col='PassengerId') 
titanic.head()

Po wstępnej analizie wczytanych danych można dojść do wniosku, iż kolumna **Ship** posiada tylko jedną wartość, którą jest 'Titanic'. Wydaje się, że ta kolumna nie będzie potrzebna w dalszej analizie, ponieważ analizujemy dane tylko i wyłącznie ze statku Titanic. Najrozsądniejszym wyborem będzie usunięcie tej kolumny ze zbioru.

In [238]:
clear = titanic.drop(columns='ship')
clear.head()

W kolumnie **Sex** występuje sporo błędów, które potocznie można określić jako literówki. Aby uporządkować dane w tej kolumnie należy najpierw odczytać wszystkie unikalne wartości, które znajdują się tej tabeli. 

In [219]:
types = clear['Sex'].unique() 
print(types)

Zauważyć można, iż rzeczywiście występują tak zwane literówki. Można łatwo dostrzec, które z podanych wartości powinny mieć wartość 'male', a które 'female' i zmienić je na te poprawne.

In [219]:
clear['Sex'] = clear['Sex'].replace(['femmale'],'female') 
clear['Sex'] = clear['Sex'].replace(['fem'],'female') 
clear['Sex'] = clear['Sex'].replace(['mal'],'male') 
clear['Sex'] = clear['Sex'].replace(['malef'],'male') 
types = clear['Sex'].unique() 
print(types)

Kolejnym krokiem jest sprawdzenie czy kolumny w zbiorze danych posiadają puste wartości.

In [236]:
titanic.isnull().sum()

Zastosowana funkcja zwróciła nam informację o zsumowanej ilości pustych (NULL) danych. Danych brakuje w trzech kolumnach: **Age**, **Cabin** oraz **Embarked**, w których brakuje odpowiednio 173, 685, 2 wartości.

Kolejnym krokiem jest sprawdzenie typów danych kolumn w zbiorze danych.

In [237]:
titanic.dtypes

Z wyniku działania funkcji można zwrócić uwagę na typ tabeli **Age**, która posiadaja typ danych **object**. Natomiast jest to kolumna, która powinny zawierać tylko i wyłącznie liczby, co może świadczyć o tym, że w tej kolumnie występują dane nieprawidłowe.

In [219]:

for c, row in clear.iterrows():
    try:
        clear.at[c, 'Age'] = float(row['Age'])       
    except ValueError:
        clear.at[c, 'Age'] = np.NaN

clear[['Age']] = clear[['Age']].astype(float).round(2)
clear.dtypes


Kolejnym krokiem jest oczyszczenie kolumny **Age**, która jak było analizowane wcześniej, posiada puste dane. Dane trzeba uzupełnić. Najlepszą metodą będzie uzupełnienie danych średnią wartością kolumny.

In [196]:
clear['Age'] = clear['Age'].fillna(round(clear['Age'].mean()))
for index, row in clear.iterrows():
    print(row["Age"])


Kontynuując analizę zbioru można dostrzec również, że tak jak kolumna **Ship**, tak samo kolumna **Cabin** nie posiada żadnych użytecznych danych, które mogłyby później zostać użyte w analizie, dlatego kolumna ta zostanie usunięta.

In [247]:
clear.drop('Cabin', axis = 1, inplace = True) 
clear.head() 


Kolumna **Embarked** posiada 2 puste rekordy jak można było zobaczyć na początku analizy. Należy je uzupełnić. W tym celu została znaleziona najczęściej występująca wartość, która została wpisana w brakujących rekordach

In [246]:
clear['Embarked'].value_counts().idxmax() 


Kolumna **Embarked** została uzupełniona wartością 'S'.

In [246]:
fill = clear['Embarked'].value_counts().idxmax() 
clear['Embarked'] = clear['Embarked'].fillna('fill') 


Ostatnim krokiem po oczyszczeniu zbioru danych jest zapisanie oczyszczonego zbioru jako plik o nazwie **TitanicCleaned** o rozszerzeniu **.tsv**.

In [251]:
clear.to_csv('TitanicCleaned.tsv', sep = '\t')

W celu weryfikacji czy wszystkie dane zostały uzupełnione, wczytany zostanie wcześniej zapisany plik z oczyszczonym zbiorem danych. Zostanie również wywołana funkcja odczytująca czy znajdują się jakiekolwiek puste dane w całym zbiorze.

In [235]:
titanic2 = pd.read_csv('TitanicCleaned.tsv', sep='\t', index_col='PassengerId') 
titanic2.head()

In [235]:
titanic2.isnull().sum() 


Jak można zauważyć, wszystkie wartości w zbiorze danych zostały uzupełnione.